**Import Packages**

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import cifar100
import numpy as np
import matplotlib.pyplot as plt

**Prepare Data**

*Load Data*

In [2]:
#(X_train,y_train),(X_test,y_test) = mnist.load_data()
#(X_train,y_train),(X_test,y_test) = fashion_mnist.load_data()
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 28s 0us/step


*Normalize Data*

In [3]:
X_train = X_train/255
X_test = X_test/255

**Create Autoencoder**

*Encoder Model*

In [4]:
encoder = Sequential()
#encoder.add(Flatten(input_shape=[28,28]))
#encoder.add(Dense(units=400,activation='relu'))
#encoder.add(Dense(units=200,activation='relu'))
#encoder.add(Dense(units=100,activation='relu'))
#encoder.add(Dense(units=50,activation='relu'))
#encoder.add(Dense(units=25,activation='relu'))
encoder.add(Flatten(input_shape=[32,32,3]))
encoder.add(Dense(units=1200,activation='relu'))
encoder.add(Dense(units=600,activation='relu'))
encoder.add(Dense(units=300,activation='relu'))
encoder.add(Dense(units=150,activation='relu'))
encoder.add(Dense(units=75,activation='relu'))

*Decoder Model*

In [5]:
decoder = Sequential()
#decoder.add(Dense(units=50,input_shape=[25],activation='relu'))
#decoder.add(Dense(units=100,activation='relu'))
#decoder.add(Dense(units=200,activation='relu'))
#decoder.add(Dense(units=400,activation='relu'))
#decoder.add(Dense(units=784,activation='sigmoid'))
#decoder.add(Reshape([28,28]))
decoder.add(Dense(units=150,input_shape=[75],activation='relu'))
decoder.add(Dense(units=300,activation='relu'))
decoder.add(Dense(units=600,activation='relu'))
decoder.add(Dense(units=1200,activation='relu'))
decoder.add(Dense(units=32*32*3,activation='sigmoid'))
decoder.add(Reshape([32,32,3]))

*Autoencoder Model*

In [6]:
autoencoder = Sequential([encoder,decoder])

*Compile Autoencoder*

In [7]:
#autoencoder.compile(loss='binary_crossentropy',optimizer=SGD(lr=1.5),metrics=['accuracy'])
autoencoder.compile(loss='mean_squared_error',optimizer=SGD(lr=1.5),metrics=['mean_squared_error'])

*Train Aautoencoder*

In [8]:
#autoencoder.fit(X_train,X_train,epochs=100,validation_data=(X_test,X_test))
X_train_ec = X_train*1
X_test_ec = X_test*1
for i in range(0,1):
    autoencoder.fit(X_train,X_train_ec,epochs=100,validation_data=(X_test,X_test_ec))
    X_train_ec = X_train + (X_train - autoencoder.predict(X_train))
    X_test_ec = X_test + (X_test - autoencoder.predict(X_test))

Epoch 1/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0618 - mean_squared_error: 0.0618 - val_loss: 0.0592 - val_mean_squared_error: 0.0592
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0433 - mean_squared_error: 0.0433 - val_loss: 0.0634 - val_mean_squared_error: 0.0634
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0379 - mean_squared_error: 0.0379 - val_loss: 0.0413 - val_mean_squared_error: 0.0413
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0338 - mean_squared_error: 0.0338 - val_loss: 0.0324 - val_mean_squared_error: 0.0324
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0327 - mean_squared_error: 0.0327 - val_loss: 0.0335 - val_mean_squared_error: 0.0335
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0322 - mean_squared_error: 0.0322 - val_loss: 0.0316 - val_mean_squared_error: 0.0316
Epoch 7/10

**Lossless Recovery**

In [9]:
#for ii in range(0,10000,1):
for ii in range(0,1,1):

    # Intialization
    x0 = X_test[ii:ii+1]*1
    y0 = autoencoder.predict(x0[:1])
    xu0 = tf.cast(x0*255, tf.uint8)
    yu0 = tf.cast(y0*255, tf.uint8)

    # Compensation-Mask-1
    x = y0*1
    y = autoencoder.predict(x[:1])
    ex0 = x0-x
    exu0 = tf.cast(ex0*255, tf.int16)
    ey0 = y0-y
    eyu0 = tf.cast(ey0*255, tf.int16)
    eyu = eyu0*1

    for k in range(1,10+1):
        for i in range(0,32,1):
            for j in range(0,32,1):
                for m in range(0,3,1):
                    #compensation-mask-1
                    if (eyu[0][i][j][m]!=0):
                        x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.1
                        #x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.01
        y = autoencoder.predict(x[:1])
        ex = x0-x
        ey = y0-y
        exu = tf.cast(ex*255, tf.int16)            
        eyu = tf.cast(ey*255, tf.int16)            

    for i in range(0,32,1):
        for j in range(0,32,1):
            for m in range(0,3,1):
                sum = x[0][i][j][m]
                if (sum<0):
                    x[0][i][j][m] = 0
                elif (sum>1):
                    x[0][i][j][m] = 1

    xu = tf.cast(x*255, tf.uint8)       
    yu = tf.cast(y*255, tf.uint8) 
    print(tf.image.psnr(xu,xu0,255))
    print(tf.image.psnr(yu0,xu0,255))
    print(tf.image.ssim(xu,xu0,255,32))
    print(tf.image.ssim(yu0,xu0,255,32))

    # Compensation-Mask-2
    x = x*1
    y = autoencoder.predict(x[:1])
    ex0 = x0-x
    exu0 = tf.cast(ex0*255, tf.int16)
    ey0 = y0-y
    eyu0 = tf.cast(ey0*255, tf.int16)
    eyu = eyu0*1
    for k in range(1,10+1):
        for i in range(0,32,1):
            for j in range(0,32,1):
                for m in range(0,3,1):
                    #compensation-mask-2
                    if (eyu[0][i][j][m]==0):
                        x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.1
                        #x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.01
        y = autoencoder.predict(x[:1])
        ex = x0-x
        ey = y0-y
        exu = tf.cast(ex*255, tf.int16)            
        eyu = tf.cast(ey*255, tf.int16)            

    for i in range(0,32,1):
        for j in range(0,32,1):
            for m in range(0,3,1):
                sum = x[0][i][j][m]
                if (sum<0):
                    x[0][i][j][m] = 0
                elif (sum>1):
                    x[0][i][j][m] = 1

    xu = tf.cast(x*255, tf.uint8)       
    yu = tf.cast(y*255, tf.uint8) 
    print(tf.image.psnr(xu,xu0,255))
    print(tf.image.psnr(yu0,xu0,255))
    print(tf.image.ssim(xu,xu0,255,32))
    print(tf.image.ssim(yu0,xu0,255,32))

    # Compensation-Nonmask
    x = x*1
    y = autoencoder.predict(x[:1])
    ex0 = x0-x
    exu0 = tf.cast(ex0*255, tf.int16)
    ey0 = y0-y
    eyu0 = tf.cast(ey0*255, tf.int16)
    eyu = eyu0*1
    for k in range(1,10+1):
        for i in range(0,32,1):
            for j in range(0,32,1):
                for m in range(0,3,1):
                    #compensation-nonmask
                    x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.00001
                    #x[0][i][j][m] = x0[0][i][j][m] + (y0[0][i][j][m]-y[0][i][j][m])*0.000001
        y = autoencoder.predict(x[:1])
        ex = x0-x
        ey = y0-y
        exu = tf.cast(ex*255, tf.int16)            
        eyu = tf.cast(ey*255, tf.int16)            

    for i in range(0,32,1):
        for j in range(0,32,1):
            for m in range(0,3,1):
                sum = x[0][i][j][m]
                if (sum<0):
                    x[0][i][j][m] = 0
                elif (sum>1):
                    x[0][i][j][m] = 1

    xu = tf.cast(x*255, tf.uint8)       
    yu = tf.cast(y*255, tf.uint8) 
    print(tf.image.psnr(xu,xu0,255))
    print(tf.image.psnr(yu0,xu0,255))
    print(tf.image.ssim(xu,xu0,255,32))
    print(tf.image.ssim(yu0,xu0,255,32))

    # Output
    #plt.imshow(x0[0])
    #plt.imshow(x[0])

tf.Tensor([35.702118], shape=(1,), dtype=float32)
tf.Tensor([19.567617], shape=(1,), dtype=float32)
tf.Tensor([0.99109226], shape=(1,), dtype=float32)
tf.Tensor([0.3766794], shape=(1,), dtype=float32)
tf.Tensor([58.595924], shape=(1,), dtype=float32)
tf.Tensor([19.567617], shape=(1,), dtype=float32)
tf.Tensor([0.9999745], shape=(1,), dtype=float32)
tf.Tensor([0.3766794], shape=(1,), dtype=float32)
tf.Tensor([inf], shape=(1,), dtype=float32)
tf.Tensor([19.567617], shape=(1,), dtype=float32)
tf.Tensor([1.], shape=(1,), dtype=float32)
tf.Tensor([0.3766794], shape=(1,), dtype=float32)
